In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Olist').getOrCreate()

25/07/23 04:02:15 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
!hadoop fs -ls -h '/data/olist'

Found 9 items
-rw-r--r--   2 geurecruitanandraj hadoop      8.6 M 2025-07-13 12:45 /data/olist/olist_customers_dataset.csv
-rw-r--r--   2 geurecruitanandraj hadoop     58.4 M 2025-07-13 12:45 /data/olist/olist_geolocation_dataset.csv
-rw-r--r--   2 geurecruitanandraj hadoop     14.7 M 2025-07-13 12:45 /data/olist/olist_order_items_dataset.csv
-rw-r--r--   2 geurecruitanandraj hadoop      5.5 M 2025-07-13 12:45 /data/olist/olist_order_payments_dataset.csv
-rw-r--r--   2 geurecruitanandraj hadoop     13.8 M 2025-07-13 12:45 /data/olist/olist_order_reviews_dataset.csv
-rw-r--r--   2 geurecruitanandraj hadoop     16.8 M 2025-07-13 12:45 /data/olist/olist_orders_dataset.csv
-rw-r--r--   2 geurecruitanandraj hadoop      2.3 M 2025-07-13 12:45 /data/olist/olist_products_dataset.csv
-rw-r--r--   2 geurecruitanandraj hadoop    170.6 K 2025-07-13 12:45 /data/olist/olist_sellers_dataset.csv
-rw-r--r--   2 geurecruitanandraj hadoop      2.6 K 2025-07-13 12:45 /data/olist/product_category_name_tran

In [4]:
hdfs_path = '/data/olist/'

In [5]:
products_df = spark.read.csv(hdfs_path+'olist_products_dataset.csv',header=True,inferSchema=True)
order_df = spark.read.csv(hdfs_path+'olist_orders_dataset.csv',header = True , inferSchema = True)
item_df = spark.read.csv(hdfs_path+'olist_order_items_dataset.csv',header = True, inferSchema=True)
payment_df = spark.read.csv(hdfs_path+'olist_order_payments_dataset.csv',header = True , inferSchema = True)
reviews_df = spark.read.csv(hdfs_path+'olist_order_reviews_dataset.csv',header = True , inferSchema = True)
location_df = spark.read.csv(hdfs_path+'olist_geolocation_dataset.csv',header = True , inferSchema = True)
seller_df = spark.read.csv(hdfs_path+'olist_sellers_dataset.csv',header = True , inferSchema = True)
trnaslation_df = spark.read.csv(hdfs_path+'product_category_name_translation.csv',header = True , inferSchema = True)

In [6]:
customers_df = spark.read.csv(hdfs_path+'olist_customers_dataset.csv',header=True, inferSchema=True)

In [7]:
# caching the most used dfs
order_df.cache()
customers_df.cache()
item_df.cache()

DataFrame[order_id: string, order_item_id: int, product_id: string, seller_id: string, shipping_limit_date: timestamp, price: double, freight_value: double]

### Joining all the dfs will be useful as customer support can get all the details from order_id or customer_id

In [8]:
order_item_joined_df = order_df.join(item_df,'order_id','inner')

In [9]:
order_items_products = order_item_joined_df.join(products_df,'product_id','inner')

In [10]:
order_item_product_seller = order_items_products.join(seller_df,'seller_id','inner')

In [12]:
full_orders_df = order_item_product_seller.join(customers_df,'customer_id','inner')

### Geolocation data , how to use it , if any data is not present in geolocation then joining the whole data will be wasted hence we would do left join as the whole data should remain. 

In [15]:
full_orders_df =  full_orders_df.join(location_df,full_orders_df.customer_zip_code_prefix == location_df.geolocation_zip_code_prefix,'left')

In [16]:
full_orders_df.show(5)

+--------------------+--------------------+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+-------------+-------------------+-----+-------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+----------------------+-----------+------------+--------------------+------------------------+-------------+--------------+---------------------------+-------------------+------------------+----------------+-----------------+
|         customer_id|           seller_id|          product_id|            order_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|order_item_id|shipping_limit_date|price|freight_value|product_category_name|product_name_lenght|pr

In [17]:
full_orders_df = full_orders_df.join(reviews_df,'order_id','left')

In [18]:
full_orders_df.show(5)

+--------------------+--------------------+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+-------------+-------------------+-----+-------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+----------------------+-----------+------------+--------------------+------------------------+-------------+--------------+---------------------------+-------------------+------------------+----------------+-----------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|            order_id|         customer_id|           seller_id|          product_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order

In [19]:
# last is payment one
full_orders_df = full_orders_df.join(payment_df,'order_id','left')

In [20]:
full_orders_df.cache()

DataFrame[order_id: string, customer_id: string, seller_id: string, product_id: string, order_status: string, order_purchase_timestamp: timestamp, order_approved_at: timestamp, order_delivered_carrier_date: timestamp, order_delivered_customer_date: timestamp, order_estimated_delivery_date: timestamp, order_item_id: int, shipping_limit_date: timestamp, price: double, freight_value: double, product_category_name: string, product_name_lenght: int, product_description_lenght: int, product_photos_qty: int, product_weight_g: int, product_length_cm: int, product_height_cm: int, product_width_cm: int, seller_zip_code_prefix: int, seller_city: string, seller_state: string, customer_unique_id: string, customer_zip_code_prefix: int, customer_city: string, customer_state: string, geolocation_zip_code_prefix: int, geolocation_lat: double, geolocation_lng: double, geolocation_city: string, geolocation_state: string, review_id: string, review_score: string, review_comment_title: string, review_commen

In [23]:
sum(10,20)

TypeError: 'int' object is not iterable

In [25]:
sum([10,20]) # sum will always take an iterable as argument

30

In [26]:
# Total Revenue per seller
from pyspark.sql.functions import *
# here sum was giving error as it would take sum inbuilt function , so to take this we have to import functions from pyspark library.
seller_review_df = full_orders_df.groupBy('seller_id').agg(sum('price').alias('Total_review'))

In [27]:
seller_review_df.show(5)

+--------------------+------------------+
|           seller_id|      Total_review|
+--------------------+------------------+
|8e6cc767478edae94...|1145757.4000000018|
|d2374cbcbb3ca4ab1...|3375517.5500000096|
|26d8a1c7c75d51304...| 1525399.500000001|
|0885aaf116795758d...|295064.90000000043|
|59fb871bf6f4522a8...|3001221.8100000056|
+--------------------+------------------+
only showing top 5 rows



### Homework to be done on own as data is cached now
1. Total orders per customer
2. Average review Score per seller
3. Most sold product (top 10)
4. Top customers by spending

In [28]:
full_orders_df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (null

In [29]:
# 1. Total orders per customer
Total_orders_per_customer_df = full_orders_df.groupBy('customer_id').agg(sum('order_id').alias('Customer_order'))

In [31]:
Total_orders_per_customer_df.show(5)

+--------------------+--------------+
|         customer_id|Customer_order|
+--------------------+--------------+
|6ec2b4682814cfdac...|          NULL|
|77311c5e39497bf57...|          NULL|
|f0031dc04ad8f5311...|          NULL|
|acac8132a5468cf70...|          NULL|
|29c37f46cdc35fd09...|          NULL|
+--------------------+--------------+
only showing top 5 rows



In [32]:
# 2. Average review Score per seller
Average_review_Score_per_seller_df = full_orders_df.groupBy('seller_id').agg(avg('review_score').alias('Avg_review_score'))

In [33]:
Average_review_Score_per_seller_df.show()

+--------------------+------------------+
|           seller_id|  Avg_review_score|
+--------------------+------------------+
|7a67c85e85bb2ce85...| 4.258920734844587|
|9d213f303afae4983...|               5.0|
|d2374cbcbb3ca4ab1...|  3.71899382437114|
|1835b56ce799e6a4d...| 3.448500717288149|
|d650b663c3b5f6fb3...| 4.276244626265428|
|aded58c8142dedc54...|2.4955357142857144|
|cca3071e3e9bb7d12...|3.8663984994689384|
|2b3e4a2a3ea8e0193...| 4.457070707070707|
|dc4a0fc896dc34b0d...| 3.657070856130951|
|92eb0f42c21942b65...|  3.67118597644481|
|f3b80352b986ab4d1...|4.1966389194283025|
|e9779976487b77c6d...|4.2127329046350255|
|2a5b78b41cd05baea...|3.9855666721338365|
|d94a40fd42351c259...| 4.104613192424153|
|f63d8f293af3a6454...| 4.640855832908813|
|d71d863e5ef30d94e...|3.0217209690893903|
|5f2684dab12e59f83...|3.8255086848635234|
|28f10b1c5e5abb9d4...|3.8086767895878526|
|4e922959ae960d389...| 3.957567696511443|
|31561f325664a8a7a...| 4.620727432077126|
+--------------------+------------

In [34]:
# 4. Top customers by spending
Top_customers_by_spending_df = full_orders_df.groupBy('customer_id')\
.agg(sum('price').alias('Top_spending_customer'))\
.orderBy('Top_spending_customer',ascending=False)

In [35]:
Top_customers_by_spending_df.show(5)

+--------------------+---------------------+
|         customer_id|Top_spending_customer|
+--------------------+---------------------+
|d3e82ccec3cb5f956...|            6662844.0|
|df55c14d1476a9a34...|            3565657.0|
|fe5113a38e3575c04...|            3293604.0|
|ec5b2ba62e5743423...|            2556120.0|
|63b964e79dee32a35...|            2501664.0|
+--------------------+---------------------+
only showing top 5 rows



In [36]:
spark.stop()